In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'/home/michele/lavoro/code/librascal/build/')

# Spherical expansion coefficients

This notebook provides examples of the kind of manipulations that need to be applied to rotate structures and spherical expansion coefficients. First, using traditional complex-spherical-harmonics tools, then, converting those to a fully real-valued pipeline

In [ ]:
from ase.io import read
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt

from rascal.representations import SphericalExpansion, SphericalInvariants
from rascal.utils import (get_radial_basis_covariance, get_radial_basis_pca, 
                          get_radial_basis_projections, get_optimal_radial_basis_hypers )
from rascal.utils import radial_basis
from rascal.utils import WDReal, CGReal

In [ ]:
# imports also some internals to demonstrate manually some CG manipulations
from rascal.utils.cg_utils import _r2c as r2c
from rascal.utils.cg_utils import _c2r as c2r
from rascal.utils.cg_utils import _cg as cg
from rascal.utils.cg_utils import _rotation as rotation
from rascal.utils.cg_utils import _wigner_d as wigner_d

## Loads the structures

In [ ]:
import urllib.request
# a collection of distorted ethanol molecules from the ANI-1 dataset 
# (see https://github.com/isayev/ANI1_dataset) with energies and forces computed using DFTB+ 
# (see https://www.dftbplus.org/)
url = 'https://raw.githubusercontent.com/cosmo-epfl/librascal-example-data/833b4336a7daf471e16993158322b3ea807b9d3f/inputs/molecule_conformers_dftb.xyz'
# Download the file from `url`, save it in a temporary directory and get the
# path to it (e.g. '/tmp/tmpb48zma.txt') in the `structures_fn` variable:
structures_fn, headers = urllib.request.urlretrieve(url)
structures_fn

In [ ]:
# Total number of structure to load
N = 100

# load the structures
frames = read(structures_fn,':{}'.format(N))

## Utility functions

Numerical evaluation of rotations and CG coefficients from sympy

In [ ]:
ml1, ml2, mL = 2,3,1

In [ ]:
mcg = cg(ml1, ml2, mL)

In [ ]:
mcg[:,:,2]

lazy storage of SPEX coefficients

In [ ]:
def relayout(hypers, x):
    # reshape spherical harmonics coefficients
    nmax = hypers['max_radial']
    lmax = hypers['max_angular']
    n = len(x)
    nel = x.shape[1]//(nmax*(lmax+1)**2)
    rx = x.reshape((n, nel, nmax, (lmax+1)**2)) # (lm) terms are stored in a compact form
    #nx = np.zeros((n, nel, nmax, lmax+1, 2*lmax+1))   # lazy layout for easier manipulation
    #for l in range(lmax+1):
    #    nx[:,:,:,l,:2*l+1] = rx[:,:,:,l**2:(l+1)**2]
    return rx

## Demonstrate the equivariance of spherical expansion coefficients

first, we compute the density expansion coefficients on a representative dataset

In [ ]:
spherical_expansion_hypers = {
    "interaction_cutoff": 3,
    "max_radial": 8,
    "max_angular": 6,
    "gaussian_sigma_constant": 0.3,
    "gaussian_sigma_type": "Constant",
    "cutoff_smooth_width": 0.5,
    "radial_basis": "DVR",
}

spex = SphericalExpansion(**spherical_expansion_hypers)

In [ ]:
selframe = frames[38]
sell = 3
feats = spex.transform(selframe).get_features(spex)
feats *= np.sqrt(len(feats))/np.linalg.norm(feats)
rfeats = relayout(spherical_expansion_hypers, feats)

In [ ]:
# random rotation in terms of Euler angles
abc = np.random.uniform(size=(3))*np.pi

In [ ]:
# this is the Cartesian rotation matrix
mrot = rotation(*abc)

In [ ]:
# rotated structure and associated features
rotframe = selframe.copy()
rotframe.positions = rotframe.positions @ mrot.T
rotframe.cell = rotframe.cell @ mrot.T
rotfeats = spex.transform(rotframe).get_features(spex)
rotfeats *= np.sqrt(len(rotfeats))/np.linalg.norm(rotfeats)
rfeats_rot = relayout(spherical_expansion_hypers, rotfeats)

In [ ]:
print(rfeats[0,0,0,sell**2:(sell+1)**2])
np.linalg.norm(rfeats[0,0,0,sell**2:(sell+1)**2])

In [ ]:
print(rfeats_rot[0,0,0,sell**2:(sell+1)**2])
np.linalg.norm(rfeats_rot[0,0,0,sell**2:(sell+1)**2])

## Rotate the spherical expansion features using Wigner matrices

In [ ]:
# computing the wigner matrix takes some time for L>4
mwd = wigner_d(sell, *abc)

In [ ]:
# orthogonality
plt.matshow(np.real(np.conjugate(mwd.T)@mwd))

In [ ]:
#  back and forth to check transformation from real to complex SPH
c2r(r2c(rfeats[0,0,0,sell**2:(sell+1)**2])) - rfeats[0,0,0,sell**2:(sell+1)**2]

In [ ]:
rfeats_rot[0,0,0,sell**2:(sell+1)**2]

In [ ]:
c2r(np.conjugate(mwd)@r2c(rfeats[0,0,0,sell**2:(sell+1)**2]))

## Compute CG iteration and show that it transforms properly

basically, here we compute covariant, lambda-SOAP features by combining spherical expansion coefficients,
following the idea behind NICE [[original paper](doi.org/10.1063/5.0021116)]

In [ ]:
# these are the indices of the features 
ml1, ml2, mL = 3,2,3
mcg = cg(ml1, ml2, mL)
mwd = wigner_d(mL, *abc)

In [ ]:
cg1 = c2r(np.einsum("abc,a,b->c",mcg,
                    r2c(rfeats[0,0,0,ml1**2:(ml1+1)**2]), 
                    r2c(rfeats[0,0,0,ml2**2:(ml2+1)**2])))

In [ ]:
rotcg1 = c2r(np.einsum("abc,a,b->c",mcg,
                    r2c(rfeats_rot[0,0,0,ml1**2:(ml1+1)**2]), 
                    r2c(rfeats_rot[0,0,0,ml2**2:(ml2+1)**2])) )

In [ ]:
cg1

In [ ]:
rotcg1

In [ ]:
c2r(np.conjugate(mwd)@r2c(cg1))

## Direct real transformations

There's no "real" reason to go through the complex algebra for rotations - we can transform once and for all the coefficients and be done with that!

In [ ]:
# matrix version of the real-2-complex and complex-2-real transformations
r2c_mat = np.hstack([r2c(np.eye(2*mL+1)[i])[:,np.newaxis] for i in range(2*mL+1)])
c2r_mat = np.conjugate(r2c_mat.T)

In [ ]:
# we can use this to transform features
r2c_mat @ cg1 - r2c(cg1)

In [ ]:
# and Wigner D matrix as well
real_mwd = np.real(c2r_mat @ np.conjugate(mwd) @ r2c_mat)

The direct real rotation is equal (modulo noise) to going back and forth from complex sph

In [ ]:
real_mwd @ cg1 - rotcg1

this also applies to the CG iteration!

In [ ]:
r2c_mat_l1 = np.hstack([r2c(np.eye(2*ml1+1)[i])[:,np.newaxis] for i in range(2*ml1+1)])
r2c_mat_l2 = np.hstack([r2c(np.eye(2*ml2+1)[i])[:,np.newaxis] for i in range(2*ml2+1)])
r2c_mat_L = np.hstack([r2c(np.eye(2*mL+1)[i])[:,np.newaxis] for i in range(2*mL+1)])

In [ ]:
real_mcg = np.real(np.einsum("abc, ax, by, zc -> xyz", mcg, r2c_mat_l1, r2c_mat_l2, np.conjugate(r2c_mat_L.T)))

In [ ]:
real_cg1 = np.einsum("abc,a,b->c",real_mcg,
                    rfeats[0,0,0,ml1**2:(ml1+1)**2],
                    rfeats[0,0,0,ml2**2:(ml2+1)**2])

In [ ]:
real_cg1 - cg1

# Streamlined real-only CG iter (and transformation)

Uses the utility classes defined in rascal.utils to do all of the above (and more!)

In [ ]:
WD   = WDReal(spherical_expansion_hypers["max_angular"], *abc)
CGIR = CGReal(spherical_expansion_hypers["max_angular"])

Clebsch-Gordan coefficients class. Precomputes the coefficients and uses them to combine and transform equivariant features

In [ ]:
scale = 1e7
test_feats = [ rfeats[0,0,0,l**2:(l+1)**2] *scale  for l in range(0,5) ]
test_feats_rot = [ rfeats_rot[0,0,0,l**2:(l+1)**2]*scale  for l in range(0,5) ]

In [ ]:
t1

In [ ]:
t1 = CGIR.combine(test_feats[3], test_feats[4], 3)
t1_r = CGIR.combine(test_feats_rot[3], test_feats_rot[4], 3)

In [ ]:
test_wig.rotate(t1)

In [ ]:
t1_r

In [ ]:
t2 = CGIR.combine(t1, test_feats[3], 2)
t2_r = CGIR.combine(t1_r, test_feats_rot[3], 2)

In [ ]:
test_wig.rotate(t2)

In [ ]:
t2_r

In [ ]:
t3 = CGIR.combine(t2, test_feats[3], 1)
t3_r = CGIR.combine(t2_r, test_feats_rot[3], 1)

In [ ]:
test_wig.rotate(t3)

In [ ]:
t3_r

In [ ]:
t4 = CGIR.combine(t3, t2, 3)
t4_r = CGIR.combine(t3_r, t2_r, 3)

In [ ]:
test_wig.rotate(t4)

In [ ]:
t4_r

# Bulk calculation of features

the combination can also be done in bulk!

In [ ]:
t1_bulk = CGIR.combine(rfeats[:,:,:,3**2:4**2], rfeats[:,:,:,2**2:3**2], 3)

In [ ]:
t1_bulk.shape

Want $\lambda$-SOAP? You've got it! This computes the full set of features (many will be zeros but oh well) using CG iteration. *Note this may differ from existing definitions because of the scaling of coefficients*

In [ ]:
def lambda_soap(spx, lam, cg):
    lmax = int(np.sqrt(spx.shape[-1]))-1
    nid, nel, nmax = spx.shape[:-1]
    lsoap = np.zeros((nid, nel, nmax, lmax+1, nel, nmax, lmax+1, 2*lam+1))
    for l1 in range(lmax+1):
        for l2 in range(lmax+1):            
            lsoap[:,:,:,l1,:,:,l2] = cg.combine_einsum(spx[..., l1**2:(l1+1)**2],
                                        spx[..., l2**2:(l2+1)**2], 
                                        lam, combination_string="ian,iAN->ianAN")
    return lsoap.reshape((nid, -1, 2*lam+1))

In [ ]:
%%time
lsoap = lambda_soap(rfeats, 3, CGIR)
lsoap_rot = lambda_soap(rfeats_rot, 3, CGIR)

In [ ]:
lsoap[0,101]

In [ ]:
WD.rotate(lsoap[0,101])

In [ ]:
lsoap_rot[0,101]

# Products of features

This is useful to transform quantities that can be construed as products of spherical harmonics to a coupled form, and back. That is, if you have Y^m1_l1 Y^m2_l2 you can cast it into a series of coefficients that transform like a single Y^M_L, and back. Note that the transformation depends on the initial values of l1,l2

In [ ]:
scale = 1e7
test_feats = [ rfeats[0,0,0,l**2:(l+1)**2] *scale  for l in range(0,5) ]
test_feats_rot = [ rfeats_rot[0,0,0,l**2:(l+1)**2]*scale  for l in range(0,5) ]

test_prod = test_feats[3][:,np.newaxis]@test_feats[2][np.newaxis,:]
test_prod_rot = test_feats_rot[3][:,np.newaxis]@test_feats_rot[2][np.newaxis,:]

In [ ]:
test_coupled = CGIR.couple(test_prod)

In [ ]:
test_coupled[1]

In [ ]:
test_coupled_rot = CGIR.couple(test_prod_rot)

In [ ]:
test_coupled_rot[1][5]

In [ ]:
WD.rotate(test_coupled[1][5])

In [ ]:
test_decoupled = CGIR.decouple(test_coupled)

In [ ]:
test_prod - test_decoupled